In [1]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from dechorate import constants

# Load RIRs

In [2]:
path_to_hdf = Path('..', 'dEchorate_rirs.hdf5')
dset = h5py.File(path_to_hdf, mode='r')

signals = list(dset.keys())
signal = signals[0]
print('Signals', signals)

rooms = list(dset[signals[0]].keys())
print('Rooms', rooms)

sources = list(dset[signals[0]][rooms[0]].keys())
print(sources)

Signals ['rir']
Rooms ['000000', '000001', '000010', '000100', '001000', '010000', '011000', '011100', '011110', '011111', '020002']
['1', '2', '3', '4', '5', '6', '7', '8', '9']


In [3]:
path_to_delay_note = Path('..','data', 'dEchorate_database.csv')
df = pd.read_csv(path_to_delay_note)
df

,Unnamed: 0,filename,src_id,src_ch,src_type,src_signal,src_pos_x,src_pos_y,src_pos_z,room_code,...,mic_type,mic_id,mic_ch,mic_pos_x,mic_pos_y,mic_pos_z,array_id,array_bar_x,array_bar_y,array_bar_z
0,0,2020-01-22__22-48-02,99.0,99.0,silence,silence,NaN,NaN,NaN,10000,...,capsule,1.0,33.0,0.857713,3.909900,1.039,1.0,0.972,3.954,1.039
1,1,2020-01-22__22-48-02,99.0,99.0,silence,silence,NaN,NaN,NaN,10000,...,capsule,2.0,34.0,0.895031,3.924300,1.039,1.0,0.972,3.954,1.039
2,2,2020-01-22__22-48-02,99.0,99.0,silence,silence,NaN,NaN,NaN,10000,...,capsule,3.0,35.0,0.941679,3.942300,1.039,1.0,0.972,3.954,1.039
3,3,2020-01-22__22-48-02,99.0,99.0,silence,silence,NaN,NaN,NaN,10000,...,capsule,4.0,36.0,1.002321,3.965700,1.039,1.0,0.972,3.954,1.039
4,4,2020-01-22__22-48-02,99.0,99.0,silence,silence,NaN,NaN,NaN,10000,...,capsule,5.0,37.0,1.095616,4.001700,1.039,1.0,0.972,3.954,1.039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10907,10907,2020-01-29__17-46-25,9.0,16.0,omnidirectional,speech,0.892,3.013,1.403,20002,...,capsule,27.0,59.0,3.097404,3.396043,1.460,6.0,3.031,3.445,1.460
10908,10908,2020-01-29__17-46-25,9.0,16.0,omnidirectional,speech,0.892,3.013,1.403,20002,...,capsule,28.0,60.0,3.057159,3.425714,1.460,6.0,3.031,3.445,1.460
10909,10909,2020-01-29__17-46-25,9.0,16.0,omnidirectional,speech,0.892,3.013,1.403,20002,...,capsule,29.0,61.0,3.004841,3.464286,1.460,6.0,3.031,3.445,1.460
10910,10910,2020-01-29__17-46-25,9.0,16.0,omnidirectional,speech,0.892,3.013,1.403,20002,...,capsule,30.0,62.0,2.924352,3.523628,1.460,6.0,3.031,3.445,1.460


In [14]:
# build RIR matrix
D = len(rooms)
J = len(sources)
I = 30
Fs = constants['Fs']
L = int(2*Fs)

rirs = np.zeros([L, I, J, D])

for r, room in enumerate(rooms):

    for j, src in enumerate(sources):

        group = f'/{signal}/{room}/{src}'
        data = np.asarray(dset[group])
        
        rirs[:,:,j,r] = data[:2*Fs,:-1]
        
print(rirs.shape)

(96000, 30, 9, 11)


# RIRs to SOFAR's DRIRS

In [4]:
import sofar as sf
sf.list_conventions()

Available SOFA conventions:
SimpleFreeFieldHRTF (Version 2.0)
SingleRoomDRIR (Version 0.3)
MultiSpeakerBRIR (Version 0.3)
GeneralFIRE (Version 1.0)
FreeFieldDirectivityTF (Version 1.0)
SimpleHeadphoneIR (Version 1.0)
SimpleFreeFieldHRIR (Version 1.0)
GeneralFIR-E (Version 2.0)
SingleRoomSRIR (Version 1.0)
SimpleFreeFieldHRSOS (Version 1.0)
GeneralTF-E (Version 1.0)
GeneralFIR (Version 1.0)
SimpleFreeFieldSOS (Version 1.0)
SingleRoomMIMOSRIR (Version 1.0)
GeneralTF (Version 2.0)
FreeFieldHRTF (Version 1.0)
FreeFieldHRIR (Version 1.0)
GeneralFIR (Version 2.0)
GeneralTF (Version 1.0)



Documentation available at [sofaconventions.org](https://www.sofaconventions.org/mediawiki/index.php/SOFA_conventions)

- **SingleRoomSRIR**: spatial room impulse responses (SRIRs) measured with an arbitrary number of receivers (R>1, such as a microphone array) and an omnidirectional source (E=1) in a single room. SingleRoomSRIR is the standardized version of the previously proposed SingleRoomDRIR.
- **SingleRoomMIMOSRIR**: SRIRs measured with a compact listener containing an arbitrary number of receivers (R>1, such as a compact microphone array) and a compact source containing an arbitrary number of emitters (E>1, such as a multi-emitter loudspeaker) for multiple positions and/or orientations of the listener and/or source, in a single room.

We go for **SingleRoomMIMOSRIR**, one object for array

In [44]:
sofa = sf.Sofa('SingleRoomMIMOSRIR')
sofa.info('all')

SingleRoomMIMOSRIR 1.0 (SOFA version 2.0)
------------------------------------------
showing all entries : type (shape), flags

GLOBAL_Conventions : attribute, mandatory, read only
GLOBAL_Version : attribute, mandatory, read only
GLOBAL_SOFAConventions : attribute, mandatory, read only
    Single-room multiple-input multiple-output spatial room impulse responses, depending on Emitters
GLOBAL_SOFAConventionsVersion : attribute, mandatory, read only
GLOBAL_DataType : attribute, mandatory, read only
    Shall be FIR-E
GLOBAL_RoomType : attribute, mandatory
    Shall be 'shoebox' or 'dae'
GLOBAL_Title : attribute, mandatory
GLOBAL_DateCreated : attribute, mandatory
GLOBAL_DateModified : attribute, mandatory
GLOBAL_APIName : attribute, mandatory, read only
GLOBAL_APIVersion : attribute, mandatory, read only
GLOBAL_AuthorContact : attribute, mandatory
GLOBAL_Organization : attribute, mandatory
GLOBAL_License : attribute, mandatory
GLOBAL_ApplicationName : attribute, optional
GLOBAL_Applicati

In [48]:
room_idx = 0
print(room_idx)
arr_idx = 0
print(arr_idx)
mic_idx = np.arange(5*arr_idx,5*(arr_idx+1))
print(mic_idx)
src_idx = np.arange(0,J)
print(src_idx)
print(rirs.shape)
mimo_srir = rirs[:, mic_idx[:,None], src_idx[None,:], room_idx]
print(mimo_srir.shape)

0
0
[0 1 2 3 4]
[0 1 2 3 4 5 6 7 8]
(96000, 30, 9, 11)
(96000, 5, 9)


In [53]:
sofa = sf.Sofa('SingleRoomMIMOSRIR')
# general attirbutes
sofa.info('all')

SingleRoomMIMOSRIR 1.0 (SOFA version 2.0)
------------------------------------------
showing all entries : type (shape), flags

GLOBAL_Conventions : attribute, mandatory, read only
GLOBAL_Version : attribute, mandatory, read only
GLOBAL_SOFAConventions : attribute, mandatory, read only
    Single-room multiple-input multiple-output spatial room impulse responses, depending on Emitters
GLOBAL_SOFAConventionsVersion : attribute, mandatory, read only
GLOBAL_DataType : attribute, mandatory, read only
    Shall be FIR-E
GLOBAL_RoomType : attribute, mandatory
    Shall be 'shoebox' or 'dae'
GLOBAL_Title : attribute, mandatory
GLOBAL_DateCreated : attribute, mandatory
GLOBAL_DateModified : attribute, mandatory
GLOBAL_APIName : attribute, mandatory, read only
GLOBAL_APIVersion : attribute, mandatory, read only
GLOBAL_AuthorContact : attribute, mandatory
GLOBAL_Organization : attribute, mandatory
GLOBAL_License : attribute, mandatory
GLOBAL_ApplicationName : attribute, optional
GLOBAL_Applicati

In [71]:
print(sofa.GLOBAL_Conventions)
print(sofa.GLOBAL_SOFAConventions)
print(sofa.GLOBAL_SOFAConventionsVersion)
print(sofa.GLOBAL_DataType)
sofa.GLOBAL_RoomType = 'shoebox'
print(sofa.GLOBAL_RoomType)

title_suffix = f'room:{rooms[room_idx]}_arr:{arr_idx+1}_mics:{mic_idx.min()+1}-{mic_idx.max()+1}_src:{src_idx.min()+1}-{src_idx.max()+1}'
sofa.GLOBAL_Title = f'dEchorate_{title_suffix}'
print(sofa.GLOBAL_Title)

sofa.GLOBAL_DateCreated = '2022-04-20'
print(sofa.GLOBAL_DateCreated)
sofa.GLOBAL_DateModified = sofa.GLOBAL_DateCreated
print(sofa.GLOBAL_DateModified)

print(sofa.GLOBAL_APIName)
print(sofa.GLOBAL_APIVersion)

sofa.GLOBAL_AuthorContact = 'diego.dicarlo89@gmail.com'
print(sofa.GLOBAL_AuthorContact)

sofa.GLOBAL_Organization = 'INRIA Rennes (Fr)'
print(sofa.GLOBAL_Organization)

sofa.GLOBAL_License = 'The MIT License (MIT) Copyright (c) 2019, Diego Di Carlo. Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software. THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.'
print(sofa.GLOBAL_License)

sofa.GLOBAL_ApplicationName = 'Python'
print(sofa.GLOBAL_ApplicationName)

import platform
sofa.GLOBAL_ApplicationVersion = platform.python_version()
print(sofa.GLOBAL_ApplicationVersion)

links = {
    'website': 'http://diegodicarlo.com/datasets/dechorate',
    'data'   : 'https://zenodo.org/record/5562386',
    'code'   : 'https://github.com/Chutlhu/DechorateDB',
    'paper'  : 'https://rdcu.be/cLJIG',
}
sofa.GLOBAL_Comment = links
print(sofa.GLOBAL_Comment)

sofa.GLOBAL_RoomDescription = 'Recorded at the Bar\'Ilan acoustic lab. Refer to the publication for all the details.'
print(sofa.GLOBAL_RoomDescription)

sofa.GLOBAL_History = ''
print(sofa.GLOBAL_History)
# sofa.GLOBAL_References =
# print(sofa.GLOBAL_References)
# sofa.GLOBAL_Origin =
# print(sofa.GLOBAL_Origin)
# sofa.GLOBAL_ListenerShortName =
# print(sofa.GLOBAL_ListenerShortName)
# sofa.GLOBAL_ListenerDescription =
# print(sofa.GLOBAL_ListenerDescription)
# sofa.GLOBAL_ReceiverShortName =
# print(sofa.GLOBAL_ReceiverShortName)
# sofa.GLOBAL_ReceiverDescription =
# print(sofa.GLOBAL_ReceiverDescription)
# sofa.GLOBAL_SourceShortName =
# print(sofa.GLOBAL_SourceShortName)
# sofa.GLOBAL_SourceDescription =
# print(sofa.GLOBAL_SourceDescription)
# sofa.GLOBAL_EmitterShortName =
# print(sofa.GLOBAL_EmitterShortName)
# sofa.GLOBAL_EmitterDescription =
# print(sofa.GLOBAL_EmitterDescription)
# sofa.GLOBAL_DatabaseName =
# print(sofa.GLOBAL_DatabaseName)
# sofa.ListenerPosition =
# print(sofa.ListenerPosition)
# sofa.ListenerPosition_Type =
# print(sofa.ListenerPosition_Type)
# sofa.ListenerPosition_Units =
# print(sofa.ListenerPosition_Units)
# sofa.ListenerView =
# print(sofa.ListenerView)
# sofa.ListenerUp =
# print(sofa.ListenerUp)
# sofa.ListenerView_Type =
# print(sofa.ListenerView_Type)
# sofa.ListenerView_Units =
# print(sofa.ListenerView_Units)
# sofa.ReceiverPosition =
# print(sofa.ReceiverPosition)
# sofa.ReceiverPosition_Type =
# print(sofa.ReceiverPosition_Type)
# sofa.ReceiverPosition_Units =
# print(sofa.ReceiverPosition_Units)
# sofa.ReceiverView =
# print(sofa.ReceiverView)
# sofa.ReceiverUp =
# print(sofa.ReceiverUp)
# sofa.ReceiverView_Type =
# print(sofa.ReceiverView_Type)
# sofa.ReceiverView_Units =
# print(sofa.ReceiverView_Units)
# sofa.SourcePosition =
# print(sofa.SourcePosition)
# sofa.SourcePosition_Type =
# print(sofa.SourcePosition_Type)
# sofa.SourcePosition_Units =
# print(sofa.SourcePosition_Units)
# sofa.SourceView =
# print(sofa.SourceView)
# sofa.SourceUp =
# print(sofa.SourceUp)
# sofa.SourceView_Type =
# print(sofa.SourceView_Type)
# sofa.SourceView_Units =
# print(sofa.SourceView_Units)
# sofa.EmitterPosition =
# print(sofa.EmitterPosition)
# sofa.EmitterPosition_Type =
# print(sofa.EmitterPosition_Type)
# sofa.EmitterPosition_Units =
# print(sofa.EmitterPosition_Units)
# sofa.EmitterView =
# print(sofa.EmitterView)
# sofa.EmitterUp =
# print(sofa.EmitterUp)
# sofa.EmitterView_Type =
# print(sofa.EmitterView_Type)
# sofa.EmitterView_Units =
# print(sofa.EmitterView_Units)
# sofa.RoomCornerA =
# print(sofa.RoomCornerA)
# sofa.RoomCornerB =
# print(sofa.RoomCornerB)
# sofa.RoomCorners =
# print(sofa.RoomCorners)
# sofa.RoomCorners_Type =
# print(sofa.RoomCorners_Type)
# sofa.RoomCorners_Units =
# print(sofa.RoomCorners_Units)
# sofa.Data_IR =
# print(sofa.Data_IR)
# sofa.Data_SamplingRate =
# print(sofa.Data_SamplingRate)
# sofa.Data_SamplingRate_Units =
# print(sofa.Data_SamplingRate_Units)
# sofa.Data_Delay =
# print(sofa.Data_Delay)

print(sofa.verify())

SOFA
SingleRoomMIMOSRIR
1.0
FIR-E
shoebox
dEchorate_room:000000_arr:1_mics:1-5_src:1-9
2022-04-20
2022-04-20
sofar SOFA API for Python (pyfar.org)
0.3.1
diego.dicarlo89@gmail.com
INRIA Rennes (Fr)
The MIT License (MIT) Copyright (c) 2019, Diego Di Carlo. Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software. THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND 

ValueError: 
ERRORS
------
Detected data of wrong type:
- GLOBAL_Comment must be string but is <class 'dict'>


In [40]:
sofa.verify()